# Ink2Pixel: Image Preprocessing and Text Line Detection

This notebook demonstrates the complete pipeline for preprocessing handwritten text images and detecting text lines using contour analysis.

## Pipeline Overview
1. **Load** original handwritten image
2. **Preprocess** using CLAHE, bilateral filtering, and adaptive thresholding  
3. **Detect** text lines using horizontal projection peak detection
4. **Visualize** results with bounding boxes

## Setup and Imports

In [ ]:
import sys
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Enable inline plotting
%matplotlib inline

# Add project root to path
project_root = Path.cwd()
sys.path.append(str(project_root))

from preprocessing import ImagePreprocessor
from preprocessing.contour_analyzer import ContourAnalyzer
from preprocessing.utils import load_image, save_image

print("✓ Imports successful")
print(f"Project root: {project_root}")

## Configuration

In [ ]:
# Define file paths
input_image_path = "examples/sample_images/handwritten_note.jpg"
output_dir = "examples/output"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Check if input image exists
if not os.path.exists(input_image_path):
    print(f"Input image not found: {input_image_path}")
    print("Please add a sample image to continue.")
else:
    print(f"✓ Input image found: {input_image_path}")
    print(f"✓ Output directory: {output_dir}")

## Step 1: Load and Display Original Image

In [ ]:
# Load the original image
original_image = load_image(input_image_path)

if original_image is None:
    print("Failed to load image!")
else:
    # Save original image to output directory
    save_image(original_image, os.path.join(output_dir, "original.jpg"))
    
    # Display the original image
    plt.figure(figsize=(12, 8))
    if len(original_image.shape) == 3:
        # Convert BGR to RGB for matplotlib
        plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(original_image, cmap='gray')
    
    plt.title('Original Handwritten Image', fontsize=16, pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"✓ Image loaded successfully")
    print(f"  - Dimensions: {original_image.shape}")
    print(f"  - Data type: {original_image.dtype}")
    print(f"  - Size: {original_image.size:,} pixels")

## Step 2: Image Preprocessing

The preprocessing pipeline includes:
- **Contrast enhancement** using CLAHE (Contrast Limited Adaptive Histogram Equalization)
- **Noise reduction** with bilateral filtering to preserve edges
- **Binary conversion** using adaptive thresholding
- **Morphological cleaning** to remove small artifacts

In [ ]:
# Initialize preprocessor and process the image
preprocessor = ImagePreprocessor()
processed_image = preprocessor.preprocess(
    input_image_path, 
    os.path.join(output_dir, "preprocessed.jpg")
)

if processed_image is None:
    print("Preprocessing failed!")
else:
    # Create side-by-side comparison
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Original image
    if len(original_image.shape) == 3:
        axes[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    else:
        axes[0].imshow(original_image, cmap='gray')
    axes[0].set_title('Original Image', fontsize=14)
    axes[0].axis('off')
    
    # Preprocessed image
    axes[1].imshow(processed_image, cmap='gray')
    axes[1].set_title('Preprocessed Image (Binary)', fontsize=14)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Preprocessing completed")
    print(f"  - Output dimensions: {processed_image.shape}")
    print(f"  - Binary image with {np.count_nonzero(processed_image == 0):,} foreground pixels")
    print(f"  - Saved to: {os.path.join(output_dir, 'preprocessed.jpg')}")

## Step 3: Text Line Detection

Using horizontal projection peak detection to identify individual text lines:
- Calculate row-wise pixel sums
- Detect peaks with adaptive thresholding
- Estimate line boundaries with proper spacing

In [ ]:
# Initialize contour analyzer and find text lines
contour_analyzer = ContourAnalyzer()
text_lines = contour_analyzer.find_text_lines_peaks(processed_image)

print(f"Text line detection completed")
print(f"  - Found {len(text_lines)} text lines")
print()

# Display details about each detected line
print("Detected text lines:")
print(f"{'Line':<6} {'X':<6} {'Y':<6} {'Width':<8} {'Height':<8} {'Area':<10}")
print("-" * 50)

total_area = 0
for i, line in enumerate(text_lines):
    x, y, w, h = line['bbox']
    area = w * h
    total_area += area
    print(f"{i+1:<6} {x:<6} {y:<6} {w:<8} {h:<8} {area:<10,}")

print(f"\nTotal text area: {total_area:,} pixels")

## Step 4: Visualize Results

Display the final result with green bounding boxes around each detected text line.

In [ ]:
# Create visualization with bounding boxes
lines_image = cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR)

# Draw green bounding boxes for each detected text line
for i, line in enumerate(text_lines):
    x, y, w, h = line['bbox']
    cv2.rectangle(lines_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Add line number labels
    cv2.putText(lines_image, f'{i+1}', (x-15, y+h//2), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# Save the result
save_image(lines_image, os.path.join(output_dir, "lines_detected.jpg"))

# Create a comprehensive visualization
fig, axes = plt.subplots(1, 3, figsize=(20, 8))

# Original image
if len(original_image.shape) == 3:
    axes[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
else:
    axes[0].imshow(original_image, cmap='gray')
axes[0].set_title('1. Original Image', fontsize=14)
axes[0].axis('off')

# Preprocessed image
axes[1].imshow(processed_image, cmap='gray')
axes[1].set_title('2. Preprocessed Binary', fontsize=14)
axes[1].axis('off')

# Result with bounding boxes
axes[2].imshow(cv2.cvtColor(lines_image, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'3. Text Lines Detected ({len(text_lines)} lines)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"Results saved to: {os.path.join(output_dir, 'lines_detected.jpg')}")

## Summary and Output Files

The preprocessing pipeline has completed successfully!

In [ ]:
# Display summary statistics
print("Processing Complete!\n")

print("Summary:")
print(f"  • Input image: {input_image_path}")
print(f"  • Original dimensions: {original_image.shape}")
print(f"  • Text lines detected: {len(text_lines)}")
print(f"  • Total text area: {total_area:,} pixels")
if text_lines:
    avg_height = sum(line['bbox'][3] for line in text_lines) / len(text_lines)
    print(f"  • Average line height: {avg_height:.1f} pixels")
print()

print("Output files created:")
output_files = ['original.jpg', 'preprocessed.jpg', 'lines_detected.jpg']
for filename in output_files:
    filepath = os.path.join(output_dir, filename)
    if os.path.exists(filepath):
        size_kb = os.path.getsize(filepath) / 1024
        print(f"  ✓ {filename} ({size_kb:.1f} KB)")
    else:
        print(f"{filename} (missing)")

print(f"\nAll files saved in: {output_dir}")